<a href="https://colab.research.google.com/github/nishikaz/PlayGround/blob/master/GoogleColab/LUKEonColab_NER.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# HuggingFace Transformers から LUKE を使って NER してみる

ほぼ[このノートブック](https://github.com/studio-ousia/luke/blob/master/notebooks/huggingface_conll_2003.ipynb)の写経。自分の理解用。

In [1]:
# transformers をインストールする。LUKE は現時点で master branch でしか利用できない模様
!pip install git+https://github.com/huggingface/transformers.git

  Cloning https://github.com/huggingface/transformers.git to /tmp/pip-req-build-gfwp7z2b
  Running command git clone -q https://github.com/huggingface/transformers.git /tmp/pip-req-build-gfwp7z2b
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel metadata ... done
     |████████████████████████████████| 901kB 34.4MB/s 
     |████████████████████████████████| 3.3MB 46.9MB/s 
  Created wheel for transformers: filename=transformers-4.7.0.dev0-cp37-none-any.whl size=2259809 sha256=e3c5aa2164fdfa068aae1068bf9dbc486dba0391dc2ea51bf5fbdc6566dd05e0
  Stored in directory: /tmp/pip-ephem-wheel-cache-re6hh6io/wheels/33/eb/3b/4bf5dd835e865e472d4fc0754f35ac0edb08fe852e8f21655f
Successfully built transformers


In [2]:
!pip install seqeval==1.2.2

     |████████████████████████████████| 51kB 7.3MB/s 
  Created wheel for seqeval: filename=seqeval-1.2.2-cp37-none-any.whl size=16172 sha256=277605be937f5aafc7b6653048ef966789d8ed879b3eaa35717577b347b83019
  Stored in directory: /root/.cache/pip/wheels/52/df/1b/45d75646c37428f7e626214704a0e35bd3cfc32eda37e59e5f
Successfully built seqeval


In [3]:
import unicodedata

import numpy as np
import seqeval.metrics
import spacy
import torch
from tqdm import tqdm, trange
from transformers import LukeTokenizer, LukeForEntitySpanClassification

In [4]:
# CoNLL-2003 の testb データセットをダウンロードする
!wget https://raw.githubusercontent.com/synalp/NER/master/corpus/CoNLL-2003/eng.testb

--2021-05-15 08:46:12--  https://raw.githubusercontent.com/synalp/NER/master/corpus/CoNLL-2003/eng.testb
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 748096 (731K) [text/plain]
Saving to: ‘eng.testb’

eng.testb           100%[===================>] 730.56K  --.-KB/s    in 0.02s   

2021-05-15 08:46:12 (29.2 MB/s) - ‘eng.testb’ saved [748096/748096]



In [5]:
# モデルとトーカナイザの読み込み
model = LukeForEntitySpanClassification.from_pretrained('studio-ousia/luke-large-finetuned-conll-2003')
model.eval()
model.to('cuda')

tokenizer = LukeTokenizer.from_pretrained('studio-ousia/luke-large-finetuned-conll-2003')

Some weights of the model checkpoint at studio-ousia/luke-large-finetuned-conll-2003 were not used when initializing LukeForEntitySpanClassification: ['luke.embeddings.position_ids']
- This IS expected if you are initializing LukeForEntitySpanClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing LukeForEntitySpanClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


## テストデータの読み込み

データセットを読み込み、以下3つの key を持つ dicts に変換する `load_documents` 関数を定義する。
- words: 単語のシーケンス
- labels: 正解 NER ラベルのシーケンス ("MISC", "PER", "ORG", or "LOC")
- sentence_boundaries: 文章の区切り位置（文頭単語のシーケンス中インデックス番号）を示す

`load_examples` は文書中の文章からバッチインスタンスを作成する。バッチインスタンスには文章の中でエンティティになりうる文字位置の情報とその組み合わせを持つ。モデルは文中の推測されたエンティティ範囲に可能性のあるラベル ["NIL", "MISC", "PER", "ORG", "LOC"] のいずれかを付与する。`NIL` はエンティティ範囲でないことを示す。この関数は以下の4つのkeyを持つdictのリストを返す。

- text: 元のテキスト
- words: 単語シーケンス
- entity_spans: エンティティ候補の位置（文字単位）
- original_word_spans: entity_spans と対応する単語インデックス番号

In [6]:
def load_documents(dataset_file):
    documents = []
    words = []
    labels = []
    sentence_boundaries = []

    with open(dataset_file) as f:
        for line in f:
            line = line.rstrip()

            if line.startswith("-DOCSTART"):
                if words:
                    documents.append(dict(
                        words=words,
                        labels=labels,
                        sentence_boundaries=sentence_boundaries
                    ))
                    words = []
                    labels = []
                    sentence_boundaries = []
                continue            

            if not line:
                if not sentence_boundaries or len(words) != sentence_boundaries[-1]:
                    sentence_boundaries.append(len(words))
            else:
                items = line.split(" ")
                words.append(items[0])
                labels.append(items[-1])
        
        if words:
            documents.append(dict(
                words=words,
                labels=labels,
                sentence_boundaries=sentence_boundaries
            ))

    return documents

In [7]:
def load_examples(documents):
    examples = []
    max_token_length = 510
    max_mention_length = 30

    for document in tqdm(documents):
        words = document["words"]
        subword_lengths = [len(tokenizer.tokenize(w)) for w in words]
        total_subword_length = sum(subword_lengths)
        sentence_boundaries = document["sentence_boundaries"]

        for i in range(len(sentence_boundaries) - 1):
            sentence_start, sentence_end = sentence_boundaries[i:i+2]
            if total_subword_length <= max_token_length:
                # シーケンス長が max_token_length より短い場合、
                # 全ての文章に含まれる単語を用いる
                context_start = 0
                context_end = len(words)
            else:
                # シーケンス長が max_token_length より長い場合、
                # 文章ごとの単語を閾値上限まで用いる
                context_start = sentence_start
                context_end = sentence_end
                cur_length = sum(subword_lengths[context_start:context_end])

                # 閾値に満たないトークン長の文章は結合して用いる
                while True:
                    if context_start > 0:
                        if cur_length + subword_lengths[context_start - 1] <= max_token_length:
                            cur_length += subword_lengths[context_start - 1]
                            context_start -= 1
                        else:
                            break
                    
                    if context_end < len(words):
                        if cur_length + subword_lengths[context_end] <= max_token_length:
                            cur_length += subword_lengths[context_end]
                            context_end += 1
                        else:
                            break
                
            text = ''
            for word in words[context_start:sentence_start]:
                if word[0] == '\'' or  (len(word) == 1 and is_punctuation(word)):
                    text = text.rstrip()
                text += word
                text += ' '
            
            sentence_words = words[sentence_start:sentence_end]
            sentence_subword_lengths = subword_lengths[sentence_start:sentence_end]

            word_start_char_positions = []
            word_end_char_positions = []
            for word in sentence_words:
                if word[0] == '\'' or (len(word) == 1 and is_punctuation(word)):
                    text = text.rstrip()
                word_start_char_positions.append(len(text))
                text += word
                word_end_char_positions.append(len(text))
                text += ' '
            
            for word in words[sentence_end:context_end]:
                if word[0] == '\'' or (len(word) == 1 and is_punctuation(word)):
                    text = text.rstrip()
                text += word
                text += ' '
            text = text.rstrip()

            entity_spans = []
            original_word_spans = []
            for word_start in range(len(sentence_words)):
                for word_end in range(word_start, len(sentence_words)):
                    if sum(sentence_subword_lengths[word_start:word_end]) <= max_mention_length:
                        entity_spans.append(
                            (word_start_char_positions[word_start], word_end_char_positions[word_end])
                        )
                        original_word_spans.append(
                            (word_start, word_end + 1)
                        )
            
            examples.append(dict(
                text=text,
                words=sentence_words,
                entity_spans=entity_spans,
                original_word_spans=original_word_spans
            ))

    return examples


In [8]:
def is_punctuation(char):
    cp = ord(char)
    if (cp >= 33 and cp <= 47) or (cp >= 58 and cp <= 64) or (cp >= 91 and cp <= 96) or (cp >= 123 and cp <= 126):
        return True
    
    cat = unicodedata.category(char)
    if cat.startswith('P'):
        return True

    return False

In [9]:
test_documents = load_documents('eng.testb')
test_examples = load_examples(test_documents)

100%|██████████| 231/231 [00:04<00:00, 51.07it/s]


## NERする

In [10]:
batch_size = 2
all_logits = []

for batch_start_idx in trange(0, len(test_examples), batch_size):
    batch_examples = test_examples[batch_start_idx:batch_start_idx + batch_size]
    texts = [example['text'] for example in batch_examples]
    entity_spans = [example['entity_spans'] for example in batch_examples]

    inputs = tokenizer(texts, entity_spans=entity_spans, return_tensors='pt', padding=True)
    inputs = inputs.to('cuda')
    with torch.no_grad():
        outputs = model(**inputs)
    all_logits.extend(outputs.logits.tolist())

100%|██████████| 1727/1727 [09:53<00:00,  2.91it/s]


In [14]:
final_labels = [label for document in test_documents for label in document['labels']]

final_predictions = []
for example_index, example in enumerate(test_examples):
    logits = all_logits[example_index]
    max_logits = np.max(logits, axis=1)
    max_indices = np.argmax(logits, axis=1)
    original_spans = example['original_word_spans']
    predictions = []
    for logit, index, span in zip(max_logits, max_indices, original_spans):
        if index != 0:
            # spanがNILでない場合
            predictions.append((logit, span, model.config.id2label[index]))
    
    # BIOラベルのシーケンスを作成する
    predicted_sequence = ['O'] * len(example['words'])
    for _, span, label in sorted(predictions, key=lambda o: o[0], reverse=True):
        if all([o == 'O' for o in predicted_sequence[span[0]:span[1]]]):
            predicted_sequence[span[0]] = 'B-' + label
            if span[1] - span[0] > 1:
                predicted_sequence[span[0]+1:span[1]] = ['I-' + label] * (span[1] - span[0] - 1)
    
    final_predictions += predicted_sequence

In [15]:
print(seqeval.metrics.classification_report([final_labels], [final_predictions], digits=4))

              precision    recall  f1-score   support

         LOC     0.9558    0.9478    0.9518      1666
        MISC     0.8553    0.8688    0.8620       701
         ORG     0.9287    0.9496    0.9391      1647
         PER     0.9683    0.9719    0.9701      1602

   micro avg     0.9386    0.9453    0.9420      5616
   macro avg     0.9270    0.9345    0.9307      5616
weighted avg     0.9389    0.9453    0.9421      5616



## 任意の文章でNER

In [18]:
nlp = spacy.load('en_core_web_sm')

def ner(text):
    doc = nlp(text)

    entity_spans = []
    original_word_spans = []
    for token_start in doc:
        for token_end in doc[token_start.i:]:
            entity_spans.append((token_start.idx, token_end.idx + len(token_end)))
            original_word_spans.append((token_start.i, token_end.i+1))

    inputs = tokenizer(text, entity_spans=entity_spans, return_tensors='pt', padding=True)
    inputs = inputs.to('cuda')
    with torch.no_grad():
        outputs = model(**inputs)

    logits = outputs.logits
    max_logits, max_indices = logits[0].max(dim=1)

    predictions = []
    for logit, index, span in zip(max_logits, max_indices, original_word_spans):
        if index != 0:
            predictions.append((logit, span, model.config.id2label[int(index)]))

    # BIOラベルのシーケンスを作成する
    predicted_sequence = ['O'] * len(doc)
    for _, span, label in sorted(predictions, key=lambda o: o[0], reverse=True):
        if all([o == 'O' for o in predicted_sequence[span[0]:span[1]]]):
            predicted_sequence[span[0]] = 'B-' + label
            if span[1] - span[0] > 1:
                predicted_sequence[span[0]+1:span[1]] = ['I-' + label] * (span[1]-span[0]-1)

    for token, label in zip(doc, predicted_sequence):
        print(token, label)

In [22]:
text1 = 'Neon Genesis Evangelion is one of my favorite anime.'
text2 = 'NH10, Boeing 777 was approved to land 34R runway of Narita Airport.'
text3 = 'Kotaro Tatsumi is a producer of Franchouchou'

ner(text1)
print('-' * 64)
ner(text2)
print('-' * 64)
ner(text3)

Neon B-MISC
Genesis I-MISC
Evangelion I-MISC
is O
one O
of O
my O
favorite O
anime O
. O
----------------------------------------------------------------
NH10 O
, O
Boeing B-MISC
777 I-MISC
was O
approved O
to O
land O
34R O
runway O
of O
Narita B-LOC
Airport I-LOC
. O
----------------------------------------------------------------
Kotaro B-PER
Tatsumi I-PER
is O
a O
producer O
of O
Franchouchou B-MISC
